In [1]:
import os
import pandas as pd
from embedding import embeddings

main_publisher = 'Khan'
methods = ['embedd-er']
class_type = 'dct' #dct or rdfs
class_file = class_type + ".csv"

script_dir = os.path.dirname(os.path.realpath('__file__'))
data_path = os.path.join(script_dir, '../Data/' + main_publisher + '/data/')
embeddings_path = os.path.join(script_dir, '../Data/' + main_publisher + '/embeddings/')
df_chapters = pd.read_csv(data_path + 'chapters.csv', sep = '|')
df_chapters['Title'] = df_chapters['Title'].astype(str)
df_chapters['Title'] = df_chapters['Title'].apply(lambda x: ' ' if x=='nan' else x)

df_chapters['Text'] = df_chapters['Text'].astype(str)
df_chapters['Text'] = df_chapters['Text'].apply(lambda x: ' ' if x=='nan' else x)

df_chapters['Type'] = df_chapters['Text'].apply(lambda x: str(type(x)))
#df_chapters[df_chapters['Text'] == ' '].head()


/home/abazouzi/Documents/Code/PrecedabilityACM/clara-datasets/lib64/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from embedding import load_model, wikipedia2vec_embedding

wikipedia2vec = load_model("../Models/enwiki_20180420_300d.txt", 300)

In [16]:
# For both BERT & EMBEDD-ER
df_chapters_embeddings = embeddings(df_chapters.Text.values, df_chapters.Cid.values, methods, main_publisher, wikipedia2vec, save = False)
df_chapters_embeddings.head()

Loading 300 model ...
Loaded 300 model ...
0 not found
1 not found
2 not found
3 not found
4 not found
5 not found
6 not found
7 not found
8 not found
9 not found
10 not found
11 not found
12 not found
13 not found
14 not found
15 not found
16 not found
17 not found
18 not found
19 not found
20 not found
21 not found
22 not found
23 not found
24 not found
25 not found
26 not found
27 not found
28 not found
29 not found
30 not found
31 not found
32 not found
33 not found
34 not found
35 not found
36 not found
38 not found
39 not found
40 not found
41 not found
42 not found
43 not found
44 not found
45 not found
46 not found
47 not found
48 not found
49 not found
50 not found
51 not found
52 not found
53 not found
54 not found
55 not found
56 not found
57 not found
58 not found
59 not found
60 not found
61 not found
62 not found
63 not found
64 not found
65 not found
66 not found
67 not found
68 not found
69 not found
70 not found
71 not found
72 not found
73 not found
74 not found
75 no

,EMBEDD-ER
0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [14]:
df_chapters_embeddings.to_csv(embeddings_path + "chapters_" + methods[0] + ".csv", sep = '|')

In [88]:
if methods[0] == 'embedd-er':
    print('Using EMBEDD-ER to embed concepts...')
    df_concepts = pd.read_csv(data_path + 'concepts.csv', sep = '|')
    df_concepts_embeddings = pd.DataFrame()
    df_concepts_embeddings['Concepts Embedding'] = df_concepts['Concept'].apply(lambda x : wikipedia2vec_embedding(wikipedia2vec, x.split('/')[-1], 300))
    #df_concepts['Concepts Embedding'] = df_concepts_embeddings
    df_concepts_embeddings.to_csv(embeddings_path + "concepts_" + methods[0] + ".csv", sep = '|')

df_concepts_embeddings.head()

Using EMBEDD-ER to embed concepts...


,Concepts Embedding
0,"[0.0484, 0.3576, -0.9482, -0.7079, -0.1311, 0...."
1,"[-0.3457, -0.0463, -0.5313, -0.0684, -0.4088, ..."
2,"[-0.1308, 0.36, -0.9123, -0.1759, -1.0825, -0...."
3,"[-0.4643, -0.0708, 0.1259, 0.3538, -0.9151, -0..."
4,"[-0.6656, -0.5116, -0.6724, 0.3429, -0.2628, 0..."


In [95]:
if methods[0] == 'BERT':
    print('Using BERT to embed concepts...')
    df_concepts = pd.read_csv(data_path + 'concepts.csv', sep = '|')
    df_concepts['Concept'] = df_concepts['Concept'].apply(lambda x : x.split('/')[-1].replace('_', ' '))
    df_concepts_embeddings = pd.DataFrame()
    df_concepts_embeddings['Concepts Embedding'] = embeddings(df_concepts.Concept.values, df_concepts.Concept.values, methods, main_publisher, save = False)
    df_concepts_embeddings.to_csv(embeddings_path + "concepts_" + methods[0] + ".csv", sep = '|')
    df_concepts_embeddings.head()


Using BERT to embed concepts...


In [3]:
# Reading the concepts and classes files
df_concepts = pd.read_csv(data_path + 'concepts.csv', sep = '|')
df_concepts_classes = pd.read_csv(data_path + 'classes/' + class_file, sep = '|', index_col=0)
df_concepts_classes.head()

,Concept,Class
0,Antibiotic,http://dbpedia.org/resource/Category:Anti-infe...
1,Antibiotic,http://dbpedia.org/resource/Category:Antibiotics
2,Antibiotic,http://dbpedia.org/resource/Category:Bactericides
3,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Veterinar...
4,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Global_is...


In [36]:
import re
def camel_to_snake(word):
    # Use a regular expression to find uppercase letters and insert an underscore before them
    s1 = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', word)
    # Use a second regular expression to handle the case where there are multiple uppercase letters in a row
    snake_case = re.sub('([a-z0-9])([A-Z])', r'\1_\2', s1).lower()
    return snake_case.capitalize()

def split_camel_case(s):
    # Add spaces before capital letters
    s = re.sub('([a-z])([A-Z])', r'\1 \2', s)
    # Handle acronyms (consecutive capital letters followed by a lowercase letter)
    s = re.sub('([A-Z]+)([A-Z][a-z])', r'\1 \2', s)
    return s

def split_snake_case(s):
    return s.replace('_', ' ')

In [93]:
if methods[0] == 'embedd-er':
    if class_type == 'rdfs':
        print('Using EMBEDD-ER to embed RDFS classes...')
        df_concepts_classes['Class Name'] = df_concepts_classes['Class'].apply(lambda x : camel_to_snake(x.split('/')[-1].split('#')[-1]))
        df_classes_embeddings = pd.DataFrame()
        df_classes_embeddings['Embedding'] = df_concepts_classes['Class Name'].apply(lambda x : wikipedia2vec_embedding(wikipedia2vec, x, 300))
    elif class_type == 'dct':
        print('Using EMBEDD-ER to embed DCT classes...')
        df_concepts_classes['Class Name'] = df_concepts_classes['Class'].apply(lambda x : x.split('/')[-1].split(':')[-1])
        df_classes_embeddings = pd.DataFrame()
        df_classes_embeddings['Embedding'] = df_concepts_classes['Class Name'].apply(lambda x : wikipedia2vec_embedding(wikipedia2vec, x, 300))
    df_classes_embeddings.to_csv(embeddings_path + "classes/"+ class_type + '_' + methods[0] + ".csv", sep = '|', index = True)
    df_classes_embeddings.head()

Using EMBEDD-ER to embed DCT classes...


In [100]:
if methods[0] == 'BERT':
    if class_type == 'rdfs':
        print('Using BERT to embed RDFS classes...')
        df_concepts_classes['Class Name'] = df_concepts_classes['Class'].apply(lambda x : split_camel_case(x.split('/')[-1].split('#')[-1]))
        df_classes_embeddings = pd.DataFrame()
        df_classes_embeddings['Embedding'] = embeddings(df_concepts_classes['Class Name'].values, df_concepts_classes['Class Name'].values, methods, main_publisher, save = False)
    elif class_type == 'dct':
        print('Using BERT to embed DCT classes...')
        df_concepts_classes['Class Name'] = df_concepts_classes['Class'].apply(lambda x : split_snake_case(x.split('/')[-1].split(':')[-1]))
        df_classes_embeddings = pd.DataFrame()
        df_classes_embeddings['Embedding'] = embeddings(df_concepts_classes['Class Name'].values, df_concepts_classes['Class Name'].values, methods, main_publisher, save = False)
    df_classes_embeddings.to_csv(embeddings_path + "classes/"+ class_type + '_' + methods[0] + ".csv", sep = '|', index = True)
    df_classes_embeddings.head()

Using BERT to embed RDFS classes...


In [6]:
df_concepts_classes.head(20)

,Concept,Class
0,Antibiotic,http://dbpedia.org/resource/Category:Anti-infe...
1,Antibiotic,http://dbpedia.org/resource/Category:Antibiotics
2,Antibiotic,http://dbpedia.org/resource/Category:Bactericides
3,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Veterinar...
4,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Global_is...
5,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Health_di...
6,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Pharmaceu...
7,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Evolution...
8,Antimicrobial_resistance,http://dbpedia.org/resource/Category:Antimicro...
9,Ape,http://dbpedia.org/resource/Category:Apes
